In [4]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import xlrd
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric
import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
import nltk

In [3]:
!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24


In [5]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\humna\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\humna\AppData\Roaming\nltk_data...


True

In [6]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    if doc is not None:
        stop_free = " ".join([i for i in str(doc).lower().split() if i not in stop]) #remove the stopwords
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)             #remove the punctuations
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())   #normalize the text using lemmatization
        return normalized
    else:
        return doc

In [6]:
file_path = ("capstone dataset.xlsx")
data = pd.read_excel(file_path, sheet_name='Not electricians')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
electric_data = data['DESCRIPTION']
doc_clean = [clean(doc).split() for doc in electric_data] 
dictionary = corpora.Dictionary(doc_clean) 
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean] 

In [8]:
base_model_lda = gensim.models.ldamodel.LdaModel
ldamodel = base_model_lda(doc_term_matrix, 
                          id2word = dictionary, passes=50)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [9]:
def predict(new_text):
  if new_text is not None and str(new_text).strip() != '':
    cleaned_data = clean(new_text).split()
    bow_vector = dictionary.doc2bow(cleaned_data)
    filters = [lambda x: x.lower(), strip_punctuation, strip_numeric]
    for index, score in sorted(ldamodel[bow_vector], key=lambda tup: -1*tup[1]):
      res = preprocess_string(ldamodel.print_topic(index, 5), filters)
      return res
  else:
    return None

In [11]:
for d in electric_data:
  print(predict(d))

['boat', 'need', 'offer', 'custom', 'towing']
['furniture', 'review', 'removalist', 'cleaner', 'complete']
['service', 'task', 'melbourne', 'offer', 'year']
['business', 'year', 'week', 'star']
['repair', 'etc', 'pack', 'flat', 'service']
['car', 'mechanic', 'mobile', 'also', 'currently']
['service', 'move', 'happy', 'price', 'work']
['experience', 'year', 'furniture', 'car', 'tool']
['day', 'excavator', 'hire', 'bendigo', 'delivery']
['work', 'new', 'quote', 'plastering', 'year']
['type', 'issue', 'removal', 'great', 'car']
['professional', 'service', 'work', 'move', 'experience']
['clean', 'new', 'furniture', 'task', 'ute']
['free', 'experience', 'job', 'i‚äôm', 'design']
['‚ôö', 'working', 'construction', 'handyman', 'removal']
['help', 'airtasker', 'task', 'hire', 'im']
['starter', 'bosch', 'kind', 'alternator', 'motor']
['certificate', 'iii', 'building', 'construction', 'get']
['solar', 'driver', 'work', 'u', 'battery']
['shower', 'bathroom', 'move', 'renovation', 'pride']
['work'

In [12]:
coherence_model_lda = CoherenceModel(model=ldamodel, 
                                     texts=doc_clean, 
                                     dictionary=dictionary, 
                                     coherence="c_v")
coherence_lda = coherence_model_lda.get_coherence()
print('\nLDA Coherence Score: ', coherence_lda)


LDA Coherence Score:  0.39440363104521453


In [13]:
lda_model_multi = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                       id2word=dictionary,
                                       num_topics=2, 
                                       random_state=100,
                                       chunksize=100,
                                       alpha=0.01,
                                       eta=0.01,
                                       passes=5,
                                       per_word_topics=True)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [14]:
coherence_model_multicore = CoherenceModel(model=lda_model_multi, 
                                     texts=doc_clean, 
                                     dictionary=dictionary, 
                                     coherence="c_v")
coherence_multicore = coherence_model_multicore.get_coherence()
print('\nMulticore Coherence Score: ', coherence_multicore)


Multicore Coherence Score:  0.3186809136451012
